# RAG with LangChain

## Step 1. Environment Setup

This section outlines the steps taken to set up the Python environment for our project using Jupyter Notebook. We are utilizing the `pip` package manager to install a series of libraries that are essential for our project's functionality.

### 1.1 Python Package Installation

To ensure consistency and to take advantage of a faster download speed provided by the Aliyun mirror, we are using the following command to install the necessary packages:

In [1]:
!pip install langchain langchain_core langchain_community langchain_chroma pypdf faiss-cpu


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2. Indexing your file

### 2.1 Document Loading and Text Splitting for Indexing

In this section, we outline the process of loading a PDF document and splitting its text content into manageable chunks for indexing in a local knowledge base.

#### 2.1.1 Load the PDF Document

We begin by creating an instance of `PyPDFLoader` with the path to the PDF document "vitis_hls.pdf". This loader will be used to read the document's content. You can replace the document name or path as needed to index different documents in your local knowledge base.

```python
loader = PyPDFLoader("rocm-docs.pdf")  # You can change documentation here to index your local knowledge base
```

#### 2.1.2 Load Document Pages
Next, we call the load method on the loader instance to retrieve the pages of the document. This step is crucial for accessing the text content that will be processed.

```python
pages = loader.load()
```
#### 2.1.3 Initialize Text Splitter
We then initialize a RecursiveCharacterTextSplitter, which is designed to split the text based on certain separators and chunk sizes. The splitter is configured with line breaks and periods as separators, a chunk size of 1024 characters, and an overlap of 100 characters to ensure continuity between chunks.

```python
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n", "."], chunk_size=1024, chunk_overlap=100
)
```
#### 2.1.4 Split the Document into Chunks
Finally, we use the text splitter to divide the document's pages into smaller text chunks. These chunks are suitable for embedding and indexing, facilitating efficient retrieval operations.

```python
docs = text_splitter.split_documents(pages)
```

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.llms.ollama import Ollama

loader = PyPDFLoader("rocm-docs.pdf") # You can change documentation here to index your local knowledge base
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n", "."], chunk_size=1024, chunk_overlap=100
)
docs = text_splitter.split_documents(pages)


### 2.2 Document Embedding and Similarity Search with Ollama Model

In this section, we demonstrate how to perform document embedding and similarity search using the `Ollama` model. `Ollama` is a library that provides text embedding and large language model (LLM) services.

#### 2.2.1 Initialize Embedding and Language Models

First, we initialize an `OllamaEmbeddings` model for document embedding, setting the model to `nomic-embed-text`. Then, we create an `Ollama` instance connected to the remote language model service `llama3:8b`, with a context length of 8192.

```python
embedding_model = OllamaEmbeddings(model="nomic-embed-text")
llm = Ollama(base_url="http://241.4.117.29:11434", model="llama3:8b", num_ctx=8192)
```
#### 2.2.2 Create FAISS Index
Next, we construct a FAISS index using the document collection and the embedding model created above. FAISS is a library for efficient similarity search and dense vector clustering.

```python
faiss_index = FAISS.from_documents(docs, embedding_model)
```

#### 2.2.3 Perform Similarity Search
We then use the FAISS index to search for the most relevant document fragments to the query "How many burst read-writes can Vitis handle at most?" Here, we retrieve the top two most relevant results.

```python
relevant_chunk = faiss_index.similarity_search("How to install ROCm?", k=2)
```

#### 2.2.4 Print Search Results
Finally, we print the retrieved relevant information. For each retrieved document fragment, we print its page number and content.

```python
print("Here are the retrieved relevant pieces of information:")
for doc in relevant_chunk:
    print("#" * 20)
    print(f"Page {doc.metadata['page']}: {doc.page_content}")
```


In [3]:
embedding_model = OllamaEmbeddings(base_url="http://241.4.117.29:11434", model="nomic-embed-text")
llm = Ollama(base_url="http://241.4.117.29:11434", model="llama3:8b", num_ctx=8192)

faiss_index = FAISS.from_documents(docs, embedding_model)
relevant_chunk = faiss_index.similarity_search("How to install ROCm?", k=2)

print("there are relevant documentation chunks")
for doc in relevant_chunk:
    print("#"*20)
    print(str(doc.metadata["page"]) + ":", doc.page_content)

there are relevant documentation chunks
####################
25: Install onnxruntime-rocm for ROCm by following these steps.
Prerequisites
•MIGraphX is installed
•Latest PyTorch ROCm release is installed
Check Install PyTorch for ROCm for latest PIP install instructions and availability.
Important! These instructions are validated for an Ubuntu 22.04 environment. Refer to the OS support
matrix for more information.
To install MIGraphX for ONNX Runtime ,
1.Verify that the install works correctly by performing a simple inference with MIGraphX.
migraphx -driver perf --test
2.Ensure that the half package is installed.
$sudo apt install half
3.Install onnxruntime-rocm via Python PIP.
22 Chapter 2. How to guides
####################
21: Use ROCm™ on Radeon™ GPUs Documentation
Pre-requisites
•Radeon software for Linux (with ROCm) must be installed.
•MIGraphX must be installed for TensorFlow to build the correct mig execution provider.
Select installation method
Option A: TensorFlow via PIP in

## Step 3. Build RAG Chain

### 3.1 Define System Prompte
here is a guidance for how to write a good prompt: https://www.coursera.org/articles/how-to-write-chatgpt-prompts
you can write prompt in following steps:
  1. what roles will LLMs plays like
  2. relevant information
  3. what you expect LLMs do for you
  4. more instruction, i.e: generate answer in JSON format;  answer in CHINESE; 

In [6]:
from langchain_core.prompts import PromptTemplate

template = (
    "You are an AMD Vitis product expert. Please answer and provide guidance based on the user's question according to the product manual.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Please answer based on the content of the product manual.\n"
    "When answering, provide the page number(s) in the product manual where the relevant information can be found.\n"
    "If the question goes beyond the scope of the manual, clearly inform the user that the question is out of the manual's scope.\n"
    "The answer should be accurate and concise.\n"
    "---------------------\n"
    "User's question: {query_str}\n"
    "---------------------\n"
    "Answer: "
)

prompt_template = PromptTemplate.from_template(
    template,
)

print(prompt_template.format(
    context_str="Vitis™ AI software is a comprehensive AI inference development solution suitable for AMD devices, development boards, Alveo™ data center accelerator cards, selected PCs, laptops, and workstations.",
    query_str="What is Vitis?"
))

You are an AMD Vitis product expert. Please answer and provide guidance based on the user's question according to the product manual.
---------------------
Vitis™ AI software is a comprehensive AI inference development solution suitable for AMD devices, development boards, Alveo™ data center accelerator cards, selected PCs, laptops, and workstations.
---------------------
Please answer based on the content of the product manual.
When answering, provide the page number(s) in the product manual where the relevant information can be found.
If the question goes beyond the scope of the manual, clearly inform the user that the question is out of the manual's scope.
The answer should be accurate and concise.
---------------------
User's question: What is Vitis?
---------------------
Answer: 


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retrieval = faiss_index.as_retriever()

rag_chain = (
    {"context_str": retrieval | format_docs, "query_str": RunnablePassthrough()}
    | prompt_template | llm | StrOutputParser()
)

## Step 4. Test your RAG system

In [9]:
user_question = "How to install ROCm?"
output = rag_chain.invoke(user_question)
print(output)

According to the product manual, you can install ROCm by following Option A or Option B.

**Option A (PIP installation method)**: You need to have Radeon software for Linux installed with ROCm, and MIGraphX must be installed. Then, follow these steps:

1. Uninstall TensorFlow-rocm if it's already installed.
2. Install the TensorFlow-rocm wheel using pip3: `pip3 install https://repo.radeon.com/rocm/manylinux/rocm-rel-6.1.3/tensorflow_rocm-2.15.1-cp310-cp310-manylinux_2_28_x86_64.whl numpy==1.26.4`

(Refer to page 22 of the manual for more information)

**Option B (Docker installation method)**: You can install ROCm on bare metal by following the ROCm installation options, which are not covered in this specific manual.

Please note that these instructions are only validated for an Ubuntu 22.04 environment, and you should refer to the OS support matrix for more information.

**Important**: Make sure to follow the prerequisites before installing ROCm.


In [12]:
user_question = "What is ROCm?"
output = rag_chain.invoke(user_question)
print(output)

According to the product manual (page 1), "ROCm is an open-source software platform for heterogeneous computing, designed to accelerate machine learning, data analytics, and cloud computing applications on Radeon GPUs."


In [10]:
user_question = "Can I install ROCm in WSL?"
output = rag_chain.invoke(user_question)
print(output)

According to the product manual (page 5.2.1), ROCm-smi support is not available due to WSL architectural limitations for native Linux User Kernel Interface (UKI). Additionally, ROCm-profiler and Debugger are also not currently supported.

However, it's possible to run PyTorch in virtual environments with some manual updates (as described on page 5.2.2).

So, while you can't install ROCm directly in WSL, you can still use ROCm with PyTorch by following the instructions provided.


In [13]:
user_question = "Does ROCm support Tensorflow framework?"
output = rag_chain.invoke(user_question)
print(output)

According to the product manual (page 2.1.4), TensorFlow is an open-source library for solving machine-learning, deep-learning, and artificial-intelligence problems that can be used with ROCm. Specifically, it says "TensorFlow is one of the most popular and in-demand frameworks and is very active in open source contribution and development."

Additionally, on page 2.1.4, it provides information on how to install TensorFlow for ROCm using either the PIP installation method or the Docker installation method.

So, to answer your question: Yes, ROCm supports the TensorFlow framework.
